In [516]:
import pandas as pd
import numpy as np
import copy, time
import warnings
warnings.filterwarnings('ignore')
from sklearn import datasets, metrics
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

df_train = pd.read_csv('train_data.csv')
df_test = pd.read_csv('test_features.csv')

train_Y = df_train['poi']
train_Y = train_Y.astype(int)
train_num = train_Y.shape[0]


names = df_test['name']
df_train = df_train.drop(['name', 'poi'] , axis=1)
df_test = df_test.drop(['name'] , axis=1)
#df = pd.concat([df_train,df_test])
#df.reset_index(drop=True, inplace=True)

#df_train.dtypes.value_counts()


In [517]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df_train.dtypes, df_train.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df_train = df_train[num_features]
df_test = df_test[num_features]

19 Numeric Features : ['bonus', 'deferral_payments', 'deferred_income', 'director_fees', 'exercised_stock_options', 'expenses', 'from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances', 'long_term_incentive', 'other', 'restricted_stock', 'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi', 'to_messages', 'total_payments', 'total_stock_value']



In [518]:
#試一些不同填補缺值的方式（D12），搭配標準化
def min_max_normalize(x):
    x = (( (x - min(x)) / ( max(x) - min(x) ) ) - 0.5) * 2
    return x

def standard_normalize(x):
    x = ( (x - min(x)) / ( max(x) - min(x) ) )
    return x

def z_transform(x):
    x = ( (x - np.mean(x)) / (np.std(x)) )
    return x


# 空值補 0
#'salary', 'other'，空值用平均值
for feature in num_features:  
    if feature=='salary' or feature=='other':
        # 得到 median 
        mid = np.median(df_train[~df_train[feature].isnull()][feature])
        print (mid)
        #df[feature] = df[feature].fillna(df[feature].mean())
        df_train[feature] = df_train[feature].fillna(mid)
    else:
        df_train[feature] = df_train[feature].fillna(0)
    df_train[feature] = standard_normalize(df_train[feature])

#標準化
#df = StandardScaler().fit_transform(df)

    
#一起做完補缺植和標準化後，再分回去train and test
#df_train=df[:train_num]
#df_test=df[train_num:]


52382.0
251654.0


In [519]:
#自己認為outliner的features，將outliner移除

from numpy.random import seed
from numpy.random import randn
from numpy import mean
from numpy import std
print (train_Y.shape)
#outliers_index=[]        
outliner_features = [ 'other', 'total_payments']
for feature in outliner_features:
    print (feature)
    feature_data=list(df_train[feature])
    data_mean, data_std = mean(feature_data), std(feature_data)
    cut_off = data_std * 3
    lower, upper = data_mean - cut_off, data_mean + cut_off
    outliers = [x for x in feature_data if x < lower or x > upper]
    '''
    for outlier in outliers:
        #print (outlier)
        #print(df[df['bonus']==outlier].index.values)
        outliers_index.append(df[df['bonus']==outlier].index.values[0])
    outliers_removed = [x for x in bonus if x >= lower and x <= upper]
    '''

    keep_indexs = (df_train[feature]> lower) & (df_train[feature]< upper)
    df_train = df_train[keep_indexs]
    train_Y = train_Y[keep_indexs]


    print('Identified outliers: %d' % len(outliers))
    print ("data_mean: ".format(), data_mean)
    print ("data_std: ".format(),  data_std)
    #print ("cut_off" .format(),  cut_off)
    #print ("lower: " .format(),  lower)
    #print ("upper: " .format(),  upper)
    print ("outliers: " .format(),  outliers)
    #print ("outliers_index: " .format(),  outliers_index)

print (train_Y.shape)

(113,)
other
Identified outliers: 1
data_mean:  0.028326011232036503
data_std:  0.10216073156369916
outliers:  [1.0]
total_payments
Identified outliers: 3
data_mean:  0.012516408464073092
data_std:  0.016784447305329055
outliers:  [0.08384253915996144, 0.10067379142018949, 0.08118030904136705]
(109,)


In [520]:
'''
for feature in num_features:  
    if feature!='salary' and feature!='other':
        mid = np.median(df_train[~df_train[feature].isnull()][feature])
        df_train[feature]=df_train[feature].replace(0, mid)
    df_train[feature] = standard_normalize(df_train[feature])
'''

"\nfor feature in num_features:  \n    if feature!='salary' and feature!='other':\n        mid = np.median(df_train[~df_train[feature].isnull()][feature])\n        df_train[feature]=df_train[feature].replace(0, mid)\n    df_train[feature] = standard_normalize(df_train[feature])\n"

In [521]:
# 空值補 0
#'salary', 'other'，空值用中位數
for feature in num_features:  
    if feature=='salary' or feature=='other':
        # 得到 median 
        mid = np.median(df_test[~df_test[feature].isnull()][feature])
        df_test[feature] = df_test[feature].fillna(mid)
    else:
        df_test[feature] = df_test[feature].fillna(0)
        
    df_test[feature] = standard_normalize(df_test[feature])

In [522]:
import matplotlib.pyplot as plt
for col in num_features:
    plt.hist(df[col], 30)
    plt.title(col)
    plt.show()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [523]:
# 將資料最大最小化
#df = MinMaxScaler().fit_transform(df)

train_X = df_train
test_X = df_test

# 使用三種模型 : 邏輯斯迴歸 / 梯度提升機 / 隨機森林, 參數使用 Random Search 尋找
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor


lr = LogisticRegression(tol=0.001, penalty='l2', fit_intercept=True, C=1.0)
gdbt = GradientBoostingClassifier(tol=100, subsample=0.75, n_estimators=250, max_features=19,
                                  max_depth=6, learning_rate=0.03)

dt = DecisionTreeClassifier(criterion = "gini", max_depth=None, min_samples_split=2, min_samples_leaf=1)


rf = RandomForestClassifier(n_estimators=100, min_samples_split=2, min_samples_leaf=1, 
                            max_features='sqrt', max_depth=6, bootstrap=True)


#rf = RandomForestClassifier()


In [524]:
'''
# 設定要訓練的超參數組合
n_estimators = [50, 100, 150]
max_depth = [1, 3, 5, 10]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(rf, param_grid, scoring="accuracy", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(train_X, train_Y)

print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

rf_bestparam = RandomForestClassifier(n_estimators=100, min_samples_split=2, min_samples_leaf=1, 
                                      max_features='sqrt', max_depth=6, bootstrap=True)

'''


'\n# 設定要訓練的超參數組合\nn_estimators = [50, 100, 150]\nmax_depth = [1, 3, 5, 10]\nparam_grid = dict(n_estimators=n_estimators, max_depth=max_depth)\n\n## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)\ngrid_search = GridSearchCV(rf, param_grid, scoring="accuracy", n_jobs=-1, verbose=1)\n\n# 開始搜尋最佳參數\ngrid_result = grid_search.fit(train_X, train_Y)\n\nprint("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))\n\nrf_bestparam = RandomForestClassifier(n_estimators=100, min_samples_split=2, min_samples_leaf=1, \n                                      max_features=\'sqrt\', max_depth=6, bootstrap=True)\n\n'

In [525]:
# 邏輯迴歸預測檔 (結果有部分隨機, 請以 Kaggle 計算的得分為準, 以下模型同理)
lr.fit(train_X, train_Y)
lr_pred = lr.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': names, 'poi': lr_pred})
sub.to_csv('submit_lr.csv', index=False) 
cross_val_score(lr,train_X,train_Y,cv=10).mean()



0.899090909090909

In [526]:
# 梯度提升機預測檔 
gdbt.fit(train_X, train_Y)
gdbt_pred = gdbt.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': names, 'poi': gdbt_pred})
sub.to_csv('submit_gdbt.csv', index=False) 
cross_val_score(gdbt,train_X,train_Y,cv=10).mean()



0.9081818181818182

In [527]:
#決策樹預測檔
dt.fit(train_X, train_Y)
dt_pred = dt.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': names, 'poi': dt_pred})
sub.to_csv('submit_dt.csv', index=False)
cross_val_score(dt,train_X,train_Y,cv=10).mean()



0.7990909090909091

In [528]:
# 隨機森林預測檔
rf.fit(train_X, train_Y)
rf_pred = rf.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': names, 'poi': rf_pred})
sub.to_csv('submit_rf.csv', index=False) 
cross_val_score(rf,train_X,train_Y,cv=10).mean()





0.899090909090909

In [515]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(train_X, train_Y)
xgb.score(train_X, train_Y)




XGBoostError: XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed (vcomp140.dll or libgomp-1.dll for Windows, libomp.dylib for Mac OSX, libgomp.so for Linux and other UNIX-like OSes). Mac OSX users: Run `brew install libomp` to install OpenMP runtime.
  * You are running 32-bit Python on a 64-bit OS
Error message(s): ['dlopen(/Users/aliceliao/opt/anaconda3/envs/ML100Days/lib/python3.7/site-packages/xgboost/lib/libxgboost.dylib, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib\n  Referenced from: /Users/aliceliao/opt/anaconda3/envs/ML100Days/lib/python3.7/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: image not found']


In [251]:
df_temp

array([[4.09257004e-01, 2.40279096e-01, 0.00000000e+00, ...,
        5.61887092e-02, 1.07944412e-01, 7.33090455e-01],
       [3.16686967e-02, 2.40279096e-01, 9.99107995e-01, ...,
        1.11317254e-02, 4.28384543e-01, 8.23659873e-02],
       [1.77831912e-01, 2.40279096e-01, 8.68261766e-01, ...,
        1.39084385e-01, 5.67077705e-01, 3.34021342e-02],
       ...,
       [2.62469280e-01, 2.40279096e-01, 8.68261766e-01, ...,
        9.46859263e-02, 1.05057782e-02, 1.15910763e-01],
       [2.62469280e-01, 2.40279096e-01, 8.68261766e-01, ...,
        3.41902995e-02, 6.99102455e-05, 7.33976137e-02],
       [1.41291108e-01, 2.40279096e-01, 8.68261766e-01, ...,
        1.00000000e+00, 2.26064506e-01, 3.34660417e-02]])